In [ ]:
#@title { display-mode: "form" }

import requests
from bs4 import BeautifulSoup
import pandas as pd
from google.colab import files
import datetime
import pytz

# ページ範囲を指定
#@markdown - 開始ページと終了ページを記入してください．終了ページはwebサイト上で，自分で確認してください．
begin_page = 3 #@param {type:"number"} # 開始ページ
end_page = 4 #@param {type:"number"}   # 終了ページ

#@markdown - URLを入れる時は，"page=19"などのページ指定の文字列が入らないように注意してください．
#@markdown - ブラウザ上で，" https://epark.jp/dentistry/list/reg_444/?page=19 "のようになっていた場合，記入するのは，" https://epark.jp/dentistry/list/reg_444/? "までとなります．「?」などの入力漏れに注意してください．
input_URL = 'https://epark.jp/gourmet/list/reg_20/?' #@param {type:"string"}

def data_extraction(url):
    # URLを指定
    #url = "https://epark.jp/shopinfo/shk47481/"
    # リクエストを送ってHTMLコンテンツを取得
    response = requests.get(url)
    content = response.text

    # BeautifulSoupでHTMLを解析
    soup = BeautifulSoup(content, 'html.parser')

    # 店舗名を抽出
    store_name = soup.find('h1', class_='shop_name').text.strip()

    # 電話番号を抽出
    phone_tag = soup.find(lambda tag: tag.name == 'th' and '電話' in tag.text)
    if phone_tag:
        phone_number = phone_tag.find_next_sibling('td').text.strip()
    else:
        phone_number = '電話番号が見つかりません'

    # 住所を抽出
    address = soup.find('div', class_='staticMap').text.strip().split('\n')[0]

    # 定休日を抽出
    regular_holiday = soup.find('div', class_='businessHoursHoliday__text')
    if regular_holiday is not None:
        regular_holiday = regular_holiday.text.strip()
    else:
        regular_holiday = "情報なし"

    # 結果をデータフレームに整理
    data = {
        '店舗名': store_name,
        '電話番号': phone_number,
        '住所': address,
        '定休日': regular_holiday
    }
    return data

store_data = []

# ベースとなるURL
base_url = input_URL + '&page='

# 指定されたページ範囲でループ
for page in range(begin_page, end_page + 1):
    # ページごとの完全なURLを構築
    url = base_url + str(page)

    # ページの内容を取得
    response = requests.get(url)
    content = response.text

    # BeautifulSoupでHTMLを解析
    soup = BeautifulSoup(content, 'html.parser')

    # 店舗情報を含むコンテナを探す
    store_containers = soup.find_all('div', class_='result_shop_box')

    # 店舗名とhrefを抽出
    for container in store_containers:
        name = container.find('h2', class_='shop_name').text.strip()
        href = container.find('a', class_='shop_detail_link')['href']
        full_href = f'https://epark.jp{href}'
        store_data.append({'name': name, 'href': full_href})

    print(f"{page}ページ目の店舗リストを取得しました．")

number_of_shops = len(store_data)
print(f"リストは{number_of_shops}件で構成されています")
#############

results = []
count = 0

for store in store_data:
    data = data_extraction(store['href'])

    name = store['name']
    href = store['href']
    phone_number = data['電話番号']
    address = data['住所']
    regular_holiday = data['定休日']
    # 各店舗の情報を辞書としてリストに追加
    results.append({
        'name': name,
        'href': href,
        'phone_number': phone_number,
        'address': address,
        'regular_holiday': regular_holiday
    })
    count += 1
    print(f"{count}件目の情報を取得しました．")

df = pd.DataFrame(results)

jst = pytz.timezone('Asia/Tokyo')
today = datetime.datetime.now(jst).strftime("20%y年%m月%d日%H時%M分")

df.to_excel(today + '.xlsx', index=False)
files.download(today + '.xlsx')
print('ファイルの保存が完了しました．')

3ページ目の店舗リストを取得しました．
4ページ目の店舗リストを取得しました．
リストは24件で構成されています
1件目の情報を取得しました．
2件目の情報を取得しました．
3件目の情報を取得しました．
4件目の情報を取得しました．
5件目の情報を取得しました．
6件目の情報を取得しました．
7件目の情報を取得しました．
8件目の情報を取得しました．
9件目の情報を取得しました．
10件目の情報を取得しました．
11件目の情報を取得しました．
12件目の情報を取得しました．
13件目の情報を取得しました．
14件目の情報を取得しました．
15件目の情報を取得しました．
16件目の情報を取得しました．
17件目の情報を取得しました．
18件目の情報を取得しました．
19件目の情報を取得しました．
20件目の情報を取得しました．
21件目の情報を取得しました．
22件目の情報を取得しました．
23件目の情報を取得しました．
24件目の情報を取得しました．


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

ファイルの保存が完了しました．
